In [1]:
# import dependancies
import pandas as pd
import random
from datetime import datetime, timedelta
from yahoo_fin.stock_info import *
import yfinance as yf
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from splinter import Browser
from pprint import pprint
import finnhub
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

# change to your api keys 

#alphavantage_api_key = 'AJP4EKR4LE52NNK4'
alphavantage_api_key = 'CORYDT1RMNVL3DVT'
polygon_api_key = 'J9UUfkyTylf3D_s76b5pcMCnLSaKTNpD'
finnhub_client = finnhub.Client(api_key="cld35g9r01qn959jif1gcld35g9r01qn959jif20")


### Screening US Mega & Large Cap Stocks

In [3]:
file_path = '../nasdaq_screener_1700335987448.csv'

# Read the CSV file into a Pandas DataFrame
stock_list_df = pd.read_csv(file_path)

# Display the DataFrame
stock_list_df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$113.15,-1.04,-0.911%,3.310625e+10,United States,1999.0,2700770,Industrials,Electrical Products
1,AAPL,Apple Inc. Common Stock,$189.69,-0.02,-0.011%,2.950202e+12,United States,1980.0,50922657,Technology,Computer Manufacturing
2,ABBV,AbbVie Inc. Common Stock,$138.30,0.02,0.014%,2.441738e+11,United States,2012.0,4332297,Health Care,Biotechnology: Pharmaceutical Preparations
3,ABNB,Airbnb Inc. Class A Common Stock,$127.15,0.87,0.689%,8.244890e+10,United States,2020.0,3795704,Consumer Discretionary,Hotels/Resorts
4,ABT,Abbott Laboratories Common Stock,$99.55,-0.71,-0.708%,1.728246e+11,United States,NaN,6069070,Health Care,Biotechnology: Pharmaceutical Preparations


In [4]:
# create an array of tickers

ticker_list = stock_list_df['Symbol'].values

### Generating functions to retrieve stock information 

In [5]:
# define function to generate random date, and ensure that the date is a weekday, maybe start in 2014 or 2015

def generate_random_date(start_date, end_date):
    check_weekday = False
    while not check_weekday:
        # Convert the date strings to datetime objects
        start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
        end_date_dt = datetime.datetime.now()  # Using today's date as the end date

        # Calculate the range of days between the start and end date
        delta_days = (end_date_dt - start_date_dt).days

        # Generate a random number of days within the range
        random_days = random.randint(0, delta_days)

        # Calculate the random date
        random_date = start_date_dt + timedelta(days=random_days)

        # Check if the generated date is a weekday
        if random_date.weekday() < 5:
            check_weekday = True
            return random_date
        else:
            check_weekday = False



In [6]:
# define a function to choose a random stock from the list

def choose_random_stock(ticker_list):    
    
    # Randomly choose a stock from the list
    random_stock = random.choice(ticker_list)
    
    return random_stock.lower()
    

In [7]:
# define a function to check if data exists for stock on the date 

def check_stock_date(ticker, check_date):
    check_financials = False
    # Check if the input date is a Friday
    if check_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        adj_end_date = check_date + timedelta(days=3)
    else:
        # Set end date to the next day
        adj_end_date = check_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    
    
    try:
        get_data(ticker, end_date=adj_end_date)
        get_stock_financials(ticker, check_date)
        

        check_financials = True
    except:
        check_financials = False
    
    if not check_financials:
        return False
    else:
        return True
    

In [8]:
# define a function to return stock industry

def return_stock_industry(symbol):
    symbol = symbol.upper()
    # Check if the symbol is in the DataFrame
    if symbol in stock_list_df['Symbol'].values:
        # Retrieve the sector for the given symbol
        sector = stock_list_df.loc[stock_list_df['Symbol'] == symbol, 'Sector'].iloc[0]
        return sector
    else:
        return f"Symbol '{symbol}' not found in the stock list."

In [9]:

# define a function to get the adjusted closing price of a stock on a particular day

def get_stock_price(ticker, stock_date):
    # Check if the input date is a Friday
    if stock_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        price_adj_end_date = stock_date + timedelta(days=3)
    else:
        # Set end date to the next day
        price_adj_end_date = stock_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    stock_data = get_data(ticker, end_date=price_adj_end_date)
    # extracting the price 
    adj_close_price = stock_data['adjclose'].values[-1]
    
    return adj_close_price

In [10]:
# define a function to get stock trading volume

def get_stock_volume(ticker, vol_date):
    # Check if the input date is a Friday
    if vol_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        vol_adj_end_date = vol_date + timedelta(days=3)
    else:
        # Set end date to the next day
        vol_adj_end_date = vol_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    stock_data = get_data(ticker, end_date=vol_adj_end_date)
    
    #extracting the volume
    trading_vol = stock_data['volume'].values[-1]
    
    return trading_vol

In [11]:
# function to get 4 previous weekdays from given_date

def get_previous_weekdays(weekday_date):
    previous_weekdays = []
    previous_date = weekday_date - timedelta(days=1)
    while len(previous_weekdays) < 4:
        if previous_date.weekday() < 5:
            previous_weekdays.append(previous_date)
        previous_date -= timedelta(days=1)

    return previous_weekdays


In [12]:
# function to get company's financial ratios by scraping stockanalysis.com

def get_stock_financials(ticker, fin_date):
    
    financial_ratios = {}

    # setting brwoser
    browser = Browser('chrome')
    
    # URL of the website
    url = f"https://stockanalysis.com/stocks/{ticker}/financials/ratios/?p=quarterly"

    # Use Selenium to open the webpage and let JavaScript execute
    driver = webdriver.Chrome()  # Make sure you have the ChromeDriver executable in your PATH
    driver.get(url)

    # Allow some time for JavaScript to execute and load the content
    time.sleep(5)  # Adjust the sleep time based on your needs

    # Get the page source after JavaScript execution
    page_source = driver.page_source

    # Close the Selenium WebDriver
    driver.quit()

    # Parse the HTML content of the page
    soup = BeautifulSoup(page_source, "html.parser")

    table = soup.find("table", class_="w-full")

    # Check if the table is found
    try:
        # Use pandas to read the HTML table into a DataFrame
        df = pd.read_html(str(table))[0]
        # Extract date columns excluding the first column
        date_columns = df.columns[2:-1]
        
        # Convert date columns to datetime objects
        date_columns = pd.to_datetime(date_columns)
        
        # Find the closest date column to the given date that is earlier
        earlier_dates = [date for date in date_columns if date < fin_date]
        closest_date_column = max(earlier_dates)

        # Access the closest column
        closest_column = closest_date_column.strftime('%Y-%m-%d')
        
    except:
        print("Table not found on the page.")
    
    

    #extracting financial ratios
    try: 
        financial_ratios['pe_ratio']=float(df.loc[df['Quarter Ended'] == 'PE Ratio', closest_column].values[0])
        financial_ratios['debt_to_equity']=float(df.loc[df['Quarter Ended'] == 'Debt / Equity Ratio', closest_column].values[0])
        financial_ratios['quick_ratio']=float(df.loc[df['Quarter Ended'] == 'Quick Ratio', closest_column].values[0])
        total_shareholder_return_str = df.loc[df['Quarter Ended'] == 'Total Shareholder Return', closest_column].values[0]
        financial_ratios['total_shareholder_return'] = float(total_shareholder_return_str.strip('%'))
    
    except:
        pass 
    
    # URL of the website
    url = f"https://stockanalysis.com/stocks/{ticker}/financials/?p=quarterly"

    # Use Selenium to open the webpage and let JavaScript execute
    driver = webdriver.Chrome()  # Make sure you have the ChromeDriver executable in your PATH
    driver.get(url)

    # Allow some time for JavaScript to execute and load the content
    time.sleep(5)  # Adjust the sleep time based on your needs

    # Get the page source after JavaScript execution
    page_source = driver.page_source

    # Close the Selenium WebDriver
    driver.quit()

    # Parse the HTML content of the page
    soup = BeautifulSoup(page_source, "html.parser")

    # Find the table by class name
    table = soup.find("table", class_="w-full")

    # Check if the table is found
    try:
    # Use pandas to read the HTML table into a DataFrame
        df = pd.read_html(str(table))[0]
        
    except:
        print("Table not found on the page.")
    
    try:
        profit_margin_str= df.loc[df['Quarter Ended'] == 'Profit Margin', closest_column].values[0]
        financial_ratios['profit_margin'] = float(profit_margin_str.strip('%'))
        cash_margin_str= df.loc[df['Quarter Ended'] == 'Free Cash Flow Margin', closest_column].values[0]
        financial_ratios['free_cash_margin'] = float(cash_margin_str.strip('%'))
    except:
        pass

    return financial_ratios
    
    

In [13]:
# function to calculate stock's volatility

def calculate_volatility(ticker, sd_date):
    df=get_data(ticker, end_date=sd_date)
    # Calculate daily returns
    df['daily_return'] = df['adjclose'].pct_change()

    # Calculate volatility as the standard deviation of daily returns
    volatility = df['daily_return'].std()

    return volatility

In [14]:
#------------------------NEED TO FIND A NEW SOURCE---------------------------------

def get_news_sentiment(ticker, news_date):
    
    # Extract year, month, and day
    news_end_date=news_date
    end_year = news_end_date.year
    end_month = str(news_end_date.month).zfill(2)
    end_day = str(news_end_date.day).zfill(2)
    
    news_start_date = news_end_date - timedelta(weeks=4)
    start_year = news_start_date.year
    start_month = str(news_start_date.month).zfill(2)
    start_day = str(news_start_date.day).zfill(2)
   

In [15]:
# extracting daily federal interest rate information from alphavantage

# using alphavantage API
url = f'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval=daily&apikey={alphavantage_api_key}'
r = requests.get(url)
data = r.json()

daily_interest_rates = data

# converting json to dataframe
daily_interest_rate_df = pd.DataFrame(daily_interest_rates['data'])

# Export DataFrame to CSV so that we can refer to it without making an api call
daily_interest_rate_df.to_csv('daily_interest_rates.csv', index=False)


In [16]:
# extracting monthly CPI information from alphavantage

# using alphavantage api
url = f'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey=demo'
r = requests.get(url)
data = r.json()

monthly_cpi = data

# converting json to dataframe
monthly_cpi_df = pd.DataFrame(monthly_cpi['data'])
monthly_cpi_df['date'] = pd.to_datetime(monthly_cpi_df['date'])


# Export DataFrame to CSV so that we can refer to it without making an api call
monthly_cpi_df.to_csv('monthly_cpi.csv', index=False)

In [17]:
# extracting monthly unemployment rate information from alphavantage

# using alphavantage api
url = f'https://www.alphavantage.co/query?function=UNEMPLOYMENT&apikey=demo'
r = requests.get(url)
data = r.json()

monthly_unemployment_rate = data

# converting json to dataframe
monthly_unemployment_rate_df = pd.DataFrame(monthly_unemployment_rate['data'])
monthly_unemployment_rate_df['date'] = pd.to_datetime(monthly_unemployment_rate_df['date'])

# Export DataFrame to CSV so that we can refer to it without making an api call
monthly_unemployment_rate_df.to_csv('monthly_unemployment_rate.csv', index=False)

In [18]:
# define a function to extract economic indicators for the given date

def get_economic_indicators(econ_date):
    
    economic_indicators_dict={}
    
    # convert datetime object to matching string 
    formatted_date = econ_date.strftime('%Y-%m-%d')
    
    # retrieve CPI value
    # Locate the value for the given date, matching only month and year
    economic_indicators_dict['cpi'] = float(monthly_cpi_df.loc[monthly_cpi_df['date'].dt.to_period('M') == pd.to_datetime(econ_date).to_period('M'), 'value'].iloc[0])    
    
    # retrieve federal interest rate value
    economic_indicators_dict['interest_rate'] = float(daily_interest_rate_df.loc[daily_interest_rate_df['date'] == formatted_date, 'value'].iloc[0])

    # retrieve unemployement rate
    # Locate the value for the given date, matching only month and year
    economic_indicators_dict['unemployment_rate'] = float(monthly_unemployment_rate_df.loc[monthly_unemployment_rate_df['date'].dt.to_period('M') == pd.to_datetime(econ_date).to_period('M'), 'value'].iloc[0])
    
    return economic_indicators_dict

In [19]:
# function to retrieve technical indicators

def get_technical_indicators(ticker, tech_date):
    
    technical_indicators_dict={}
    
    # changing date into UNIX format
    end_timestamp = int(tech_date.timestamp())
    
    # choosing a timestamp of one week earlier
    start_tech_date = tech_date - timedelta(weeks=1)
    
    # changing the date to UNIX format
    start_timestamp = int(start_tech_date.timestamp())
    
    # retrieving SMA 
    technical_indicators_dict['sma']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='sma', indicator_fields={"timeperiod": 5})['sma'][-1]
    
    #retrieving RSI
    technical_indicators_dict['rsi']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='rsi', indicator_fields={"timeperiod": 5})['rsi'][-1]

    # retrieving EMA 
    technical_indicators_dict['ema']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='ema', indicator_fields={"timeperiod": 5})['ema'][-1]
    
    return technical_indicators_dict

In [20]:
# function to create label value for data

def create_label(ticker, y_date):
    print(y_date)
    initial_price = get_stock_price(ticker, y_date)
    print(initial_price)
    
    # find the next weekday
    
    # check if date landed on friday and calculate the next weekday date
    if y_date.weekday() == 4:
        next_weekday_date = y_date + timedelta(days=3)
    else:
        next_weekday_date = y_date + timedelta(days=1)

    print(next_weekday_date)
    
    next_price = get_stock_price(ticker, next_weekday_date)
    
    print(next_price)
    
    if next_price >= initial_price:
        return 1 # 1 is class increase, in order to simplify the classification we put stayed the same in same category as increase
    else:
        return 0 # 0 is class decrease

### Running a loop to generate multiple rows of data for training and testing

In [21]:
page=9

In [22]:
# list to store dictionaries
list_dict = []
page += 1
# runs loop until list_dic [change no. 4 as desired]
while len(list_dict) < 500:

    # data_dict
    data_dict = {}
    
    # checking to see if generated ticker-date combo has available data
    checker = False
    
    # runs this loop until checker is true
    while not checker:

        ticker = choose_random_stock(ticker_list)
        input_date = generate_random_date('2017-01-01', '2023-10-31')

        checker = check_stock_date(ticker, input_date)

    # using function to return stock industry
    data_dict['industry']=return_stock_industry(ticker)

    # retrieve weekly returns
    date = input_date
    for i in range(1, 5):
        data_dict[f'wr{i}']=(get_stock_price(ticker, date) - get_stock_price(ticker, date - timedelta(weeks = 1)))/get_stock_price(ticker, date - timedelta(weeks = 1))
        date = date - timedelta(weeks=1)

    # retrieve last 5 working day volumes
    data_dict['vol1'] = get_stock_volume(ticker, date)
    for i, weekday in enumerate(get_previous_weekdays(input_date)):
        data_dict[f'vol{i+1}'] = get_stock_volume(ticker, weekday)

    # retrieve stock financials    
    for key, value in get_stock_financials(ticker, input_date).items():
        data_dict[key] = value
    
    # retrieve stock volatility
    data_dict['volatility'] = calculate_volatility(ticker, input_date)

    # retrieve economic indicators
    for key, value in get_economic_indicators(input_date).items():
        data_dict[key] = value

    # retrieve stocktechnical indicators    
    for key, value in get_technical_indicators(ticker, input_date).items():
        data_dict[key] = value

    # generate a label for row    
    data_dict['label'] = create_label(ticker, input_date)
        
    # append dictionary to list
    list_dict.append(data_dict)
    print(len(list_dict))

2019-04-02 00:00:00
76.95865631103516
2019-04-03 00:00:00
77.38204956054688
1
2020-07-03 00:00:00
142.79833984375
2020-07-06 00:00:00
144.74400329589844
2
2020-03-18 00:00:00
198.46212768554688
2020-03-19 00:00:00
192.6221923828125
3
2023-08-29 00:00:00
370.5199890136719
2023-08-30 00:00:00
373.3999938964844
4
2020-09-16 00:00:00
130.0749969482422
2020-09-17 00:00:00
130.4854736328125
5
2021-04-21 00:00:00
136.1564483642578
2021-04-22 00:00:00
134.95884704589844
6
2021-02-19 00:00:00
490.94000244140625
2021-02-22 00:00:00
488.30999755859375
7
2019-12-31 00:00:00
47.154541015625
2020-01-01 00:00:00
47.154537200927734
8
2020-05-14 00:00:00
69.1205825805664
2020-05-15 00:00:00
69.0926284790039
9
2019-07-18 00:00:00
64.17730712890625
2019-07-19 00:00:00
64.75335693359375
10
Table not found on the page.
Table not found on the page.
Table not found on the page.
Table not found on the page.
2020-04-30 00:00:00
17.413036346435547
2020-05-01 00:00:00
17.435752868652344
11
2018-06-22 00:00:00
45

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.160)
Stacktrace:
	GetHandleVerifier [0x00007FF717A982B2+55298]
	(No symbol) [0x00007FF717A05E02]
	(No symbol) [0x00007FF7178C05AB]
	(No symbol) [0x00007FF7178A0038]
	(No symbol) [0x00007FF717926BC7]
	(No symbol) [0x00007FF71793A15F]
	(No symbol) [0x00007FF717921E83]
	(No symbol) [0x00007FF7178F670A]
	(No symbol) [0x00007FF7178F7964]
	GetHandleVerifier [0x00007FF717E10AAB+3694587]
	GetHandleVerifier [0x00007FF717E6728E+4048862]
	GetHandleVerifier [0x00007FF717E5F173+4015811]
	GetHandleVerifier [0x00007FF717B347D6+695590]
	(No symbol) [0x00007FF717A10CE8]
	(No symbol) [0x00007FF717A0CF34]
	(No symbol) [0x00007FF717A0D062]
	(No symbol) [0x00007FF7179FD3A3]
	BaseThreadInitThunk [0x00007FFBEE71257D+29]
	RtlUserThreadStart [0x00007FFBEF1EAA58+40]


In [23]:
data_dict

{'industry': 'Consumer Discretionary',
 'wr1': 0.0029039346193040654,
 'wr2': -0.031676232402648254,
 'wr3': 0.01983058758780749,
 'wr4': 0.008385645068738082,
 'vol1': 2654300,
 'vol2': 2701800,
 'vol3': 2149400,
 'vol4': 2022800}

In [24]:
print(ticker, input_date)

hlt 2017-08-21 00:00:00


In [25]:
pd.to_datetime(input_date).to_period('M')

Period('2017-08', 'M')

In [26]:
monthly_cpi_df.loc[monthly_cpi_df['date'].dt.to_period('M') == pd.to_datetime(input_date).to_period('M'), 'value'].iloc[0]  

'245.519'

In [27]:
list_dict

[{'industry': 'Energy',
  'wr1': -0.03570725937958325,
  'wr2': -0.008797658404944715,
  'wr3': 0.015402065789566101,
  'wr4': -0.0007182083211789154,
  'vol1': 2635300,
  'vol2': 2900400,
  'vol3': 2524800,
  'vol4': 1657400,
  'pe_ratio': 8.21,
  'debt_to_equity': 0.47,
  'quick_ratio': 0.76,
  'total_shareholder_return': 9.6,
  'profit_margin': 0.86,
  'free_cash_margin': -6.66,
  'volatility': 0.016354597152439317,
  'cpi': 255.548,
  'interest_rate': 2.41,
  'unemployment_rate': 3.6,
  'sma': 94.92,
  'rsi': 0,
  'ema': 95.04708333333332,
  'label': 1},
 {'industry': 'Health Care',
  'wr1': 0.05130909885037638,
  'wr2': -0.0024021839870090426,
  'wr3': -0.032075342348364094,
  'wr4': -0.06681882225420727,
  'vol1': 1247110,
  'vol2': 872418,
  'vol3': 1052838,
  'vol4': 690485,
  'pe_ratio': 44.58,
  'debt_to_equity': 0.97,
  'quick_ratio': 0.91,
  'total_shareholder_return': 1.32,
  'profit_margin': 8.36,
  'free_cash_margin': 9.83,
  'volatility': 0.02577148178177459,
  'cpi': 2

### Export data into csv

In [28]:
# remember to export data as csv
data_df = pd.DataFrame(list_dict)
data_df.to_csv(f'data_JB{page}.csv', index=False)


In [29]:
data_df

,industry,wr1,wr2,wr3,wr4,vol1,vol2,vol3,vol4,pe_ratio,...,profit_margin,free_cash_margin,volatility,cpi,interest_rate,unemployment_rate,sma,rsi,ema,label
0,Energy,-0.035707,-0.008798,0.015402,-0.000718,2635300,2900400,2524800,1657400,8.21,...,0.86,-6.66,0.016355,255.548,2.41,3.6,94.920000,0.000000,95.047083,1
1,Health Care,0.051309,-0.002402,-0.032075,-0.066819,1247110,872418,1052838,690485,44.58,...,8.36,9.83,0.025771,259.101,0.09,10.2,144.155333,100.000000,144.095925,1
2,Health Care,-0.262557,-0.049693,0.105426,-0.118102,3100800,2815300,3027000,3100200,15.93,...,3.41,3.56,0.019204,258.115,0.25,4.4,223.106667,19.984260,224.741667,0
3,Industrials,0.060226,-0.051060,-0.021225,0.006400,181700,166700,155000,408800,20.99,...,15.39,10.40,0.022123,307.026,5.33,3.8,365.180000,84.858446,366.407917,1
4,Consumer Discretionary,-0.025949,-0.052749,0.129916,-0.012914,10142600,15244600,9626600,11236400,14.15,...,4.70,7.35,0.018854,260.280,0.09,7.9,136.980000,10.717100,136.860000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,Finance,-0.033335,0.007557,0.043366,0.062641,500900,495200,439400,480300,10.18,...,-4.66,29.67,0.027220,260.388,0.09,6.9,166.130000,18.864991,164.940417,0
65,NaN,0.024719,0.015731,-0.063151,-0.027247,319500,346300,391700,248300,52.38,...,19.75,49.83,0.026436,304.127,5.08,3.7,306.270000,82.860797,305.420000,0
66,Industrials,0.159234,0.068190,-0.052825,0.037351,2094300,1021000,2165400,1062800,21.99,...,9.90,21.73,0.020355,260.229,0.09,6.7,250.503333,87.670017,251.894167,0
67,Finance,0.023803,0.020336,-0.030909,-0.046377,2208000,3115200,2639300,2454300,8.54,...,24.11,-146.19,0.023028,296.808,2.33,3.5,72.573333,48.461748,71.956250,0


In [30]:
page = 1

In [31]:
print(ticker, input_date)

hlt 2017-08-21 00:00:00
